In [1]:
#pandas（データフレーム）ライブラリ
import pandas as pd

#自動操作ライブラリ
import pyautogui as pag

#ファイルの連続読み込みライブラリ
import glob

#秒数指定型の処理待ちライブラリ
import time

In [2]:
#ファイルパスをglobで指定し、testから始まるxlsxファイルを複数読み込む
file_paths = glob.glob("/Users/kato/Desktop/shigoto/test*.xlsx")

#念の為、取得した複数ファイルの各ファイル名をプリントする
print(file_paths)

#念の為、取得した複数ファイルの個数をプリントする
print(len(file_paths))

#取得したファイルの個数をlenで数え、数値化する
#のちにdf[0]=読み込んだxlsxの0番目、1番目と個数に応じてDFを変数化するために使用する
file_num = len(file_paths)

#念の為、lenで設定した数値をプリントする
print(file_num)

['/Users/kato/Desktop/shigoto/test_1_1.xlsx', '/Users/kato/Desktop/shigoto/test_1_2.xlsx']
2
2


In [3]:
#読み込んだxlsxをfor文で順番に入れるための空のリストを作成
dfs = []

#file_numの数値が、for文でfile変数に代入される（0〜）
for file in range(file_num):
    
    #df変数にエクセルファイルを入れ、4行目をヘッダーとする
    #なお、for文で回しているためfileの数字が続く限りこの処理が繰り返される
    #globで取得したfilepathsの、file（数値）番目のエクセルをDFにするということ
    df = pd.read_excel(file_paths[file], header=4)
    
    #読み込んだDFを、dfsの空のリストに入れる
    #この状態では、dfsはlist型になっており、読み込んだDFが全て入っている
    #リスト型なので、dfs[0]やdfs[1]で確認すると一つずつのDFが確認できる
    dfs.append(df)

#range関数で、df+[i] = df[0] = dfsの0番目のような指定の仕方でDFを作成する


#df1=dfs[0]番目、df2=dfs[1]番目とする
#それぞれプリントやDF表示してみると分かる
for i in range(file_num):
    globals()["df" + str(i+1)] = dfs[i]

In [4]:
#この最初のfor文をループAとする
#この時、ループAが終わった後にループBが発生し、ループBの繰り返しが全て終わった後にループAに戻ってくる（ネスト）

#file_numの数値＝読み込んだファイル数を、変数sに代入
#これは、全てのファイルに対しループBを行うため、ファイル数分の処理をするということ

#（ループA）
for s in range(file_num):
    
    #発注数が入っていない行は削除して上書きする
    dfs[s].dropna(subset=["数量"], inplace=True)
    
    #インデックス番号がずれていると、Series番号の何番目という指定が、sの変数数値でできないので、行番号を振り直して上書きする
    dfs[s].reset_index(inplace=True)
    
    #余計なものは削除する（Seriesにする）
    #JANコードは、dfs[s]のJAN列のみを指定
    df_jan = dfs[s]["JAN"]
    
    #そのJANの個数分のデータを入力するので、そのfor分のための数値を取得する
    df_len = len(dfs[s]["JAN"])
    
    #発注数は、dfs[s]の数量列のみを指定
    df_item = dfs[s]["数量"]
    
    
    #このインデントのあるfor文をループBとする
    #（ループB）
    #for文を使って、先ほど読み込んだJANコードの個数分書き込みなど一連の処理を繰り返す
    #そのために、先ほど取得したJANコードの個数分（df_len）の数値を、変数に入れる
    for n in range(df_len):
        
        #入力画面をアクティブウィンドウにするために、所定の位置をクリックする
        pag.click(1231, 408)
        
        #JANコードのn番目を書き込む
        pag.write(str(df_jan[n]))
        pag.press("enter")
        pag.press("tab", presses=2, interval=0.01)
        
        #数量のn番目を書き込む
        pag.write(str(df_item[n]))
        pag.press("tab", presses=2, interval=0.01)
        
        #n+1が、df_lenの数値（JANの個数分の数値）と一緒になったら
        #＝全てのJANの書き込みを終え、ループBが終わったらポップアップを出す
        #その時、OKを押すとこのループBは終了し、ループAにいく
        #そして、dfs[s]の次＝2番目のDFの読み込んで書き込みに繋げる
        if n+1 == df_len:
            pag.confirm(text = "次へ行きますか？",
                              title = str(s+1)+"ファイル目が終わりました",
                              buttons = ["OK", "Calcel"])
            
        #なお、ここでOKをクリックではなく、エンターを押すだけでも次へ行ける   
        #OKを押したらTABで移動して更新（入荷登録）、日時設定、入荷先設定とかまで済ましてから次のループに行けば良いかもしれない